In [106]:
import pandas as pd
import numpy as np

In [139]:
countries_QDP_df = pd.read_csv('datasets/countries_GDP.csv', encoding = "ISO-8859-1").head(202)

columns_to_keep = ["Unnamed: 3", "Unnamed: 4"]
countries_QDP_df = countries_QDP_df[columns_to_keep]
countries_QDP_df.rename(columns={'Unnamed: 3': 'country', 'Unnamed: 4': 'gdp'}, inplace=True)
countries_QDP_df.drop([0,1,2,3], axis=0, inplace=True)
countries_QDP_df = countries_QDP_df.reset_index(drop=True)

countries_QDP_df['gdp'] = countries_QDP_df['gdp'].str.replace(',', '')
countries_QDP_df['gdp'] = countries_QDP_df['gdp'].str.replace(' ', '')


colonized_df = pd.read_csv('datasets/colonies_wikipedia.csv')
colonized_countries = list(colonized_df["colonized_country"])

colonizer_countries = list(colonized_df["colonizer_country"].value_counts().keys())
colonizer_countries_gdp = {key: 0 for key in colonizer_countries}


for index in range(len(countries_QDP_df)):
    country = countries_QDP_df.get_value(index,'country')
    if country in colonized_countries:
        colonizer_country = colonized_df.loc[colonized_df['colonized_country'] == country, 'colonizer_country'].values[0]
        country_gdp = countries_QDP_df.get_value(index,'gdp')
        colonizer_countries_gdp [colonizer_country]=colonizer_countries_gdp [colonizer_country] + int(country_gdp)
        
        countries_QDP_df.set_value(index, 'colonizer_country', colonizer_country) 
    else:
        countries_QDP_df = countries_QDP_df.drop(index, axis=0)

countries_QDP_df = countries_QDP_df.reset_index(drop=True)

In [141]:
countries_QDP_df.head()

,country,gdp,colonizer_country
0,United States,18624475,France
1,India,2263792,United Kingdom
2,Brazil,1796187,Netherlands
3,Canada,1529760,United Kingdom
4,Australia,1204616,United Kingdom


In [142]:
colonizer_countries_gdp 

{'Belgium': 11383,
 'Denmark': 20047,
 'France': 19356877,
 'Italy': 72374,
 'Netherlands': 2953139,
 'Portugal': 159935,
 'Russia': 540933,
 'Spain': 3249811,
 'United Kingdom': 8523185}